prepare llpsmd environment

Create folder and build initial file
* pdb
* psf

In [ ]:
import os
folder= 'folderName' # 
if not os.path.exists(folder):
        os.makedirs(folder)

NTag= 'N300' # 
initFile= f'packInit_{NTag}' #
# create '[initFile].pdb & [initFile].psf '

parameters
* temperature
* tiemstep (dt= 0.01)
* ion strength (ionic= 0.1)
* friction (gramma= 0.01)

In [ ]:
Temp= 300
nStep0= 100000
nStep= 50000000

Running code
* save it into `[run].py` file
* run 
```
python [run].py
```

In [ ]:
from llpsmd.utils import load_parmed
from llpsmd.omdsim import OMDSim

pmd = load_parmed(fname=folder+'/'+initFile)
print(pmd.topology)

### create simulations by opnemm add force field 
mysim = OMDSim(pmd, param= 'M2', device= 'CUDA') # device= 'CUDA', ?fname= ?
mysim.add_dh_force(ionic=0.1, temp=Temp,  n_excl=1, r_cut=4)
mysim.add_lj_lambda(eps=0.8368, n_excl=1, r_cut=4)

### attract polymers into the center for faster simulation
print('add external force, k=0.01')
mysim.add_external(k=0.01)
mysim.run_nvt(temp=Temp, gamma=0.01, dt=0.01, minimize=True, n_steps=nStep0, log_step=5000, fname=folder+'/attract')

### remove attraction and run
print('remove external force...')
mysim.remove_external()
mysim.run_nvt(temp=Temp, gamma=0.01, dt=0.01, minimize=False, n_steps=nStep, log_step=5000, fname=folder+'/output')

analysis

In [ ]:
import MDAnalysis as mda
import mdtraj as md
import numpy as np
import nglview as nv
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math
import pickle
import os

folder0= 'folderName' # !
NTag= 'N300'# ! [mutQ_]N100*
Temps= np.arange(260,380,20) # ! ( [start, stop), step )
seq= 'GY23'

### add functions

In [ ]:
# parameters calculation
# --- evoluation plot
start= 0
period= 50 # less period is more accurate, but slower
nbins= 101
len_ratio= 1 # default: 1

# --- average density profile
nFrame= 10000 # total recorded frame of simulation
timeTot= 500 # ! total time of simulation [ns]
ratio_avg= 0.5 # frame ratio for average [0-1]

end= math.floor(nFrame*len_ratio)
print(f"frame {start}:{period}:{end}")

I2= 23 # 
CA= 0.1 # [A] -> [nm]
C= 1000

In [ ]:
# loop for different temperature!
NT= np.size(Temps) 
nHf= (nbins-1)//2
denDils_pep= np.zeros(NT)
denCods_pep= np.zeros(NT)
# zHfs= np.zeros([NT,nHf])
denHfs_pep= np.zeros([NT,nHf])
denFits_pep= np.zeros([NT,nHf])
popts_pep= np.zeros([NT,4])

for i in range(NT):
    TTag= 'T'+str(Temps[i])
    folder= folder0+'/'+TTag+NTag 
    init= 'packInit_'+NTag
    u = mda.Universe(folder+'/'+init+'.psf', folder+'/output_'+TTag+'.dcd') # !
    # u = mda.Universe(seq+'/'+init+'.psf', folder+'/output_'+TTag+'.dcd')
    # record density trajectory
    sel_pep= 'not (name _NA)'
    zc, dens_pep_, frames= density_profile3(u, sel_pep, nbins, start, period, len_ratio)
    dens_pep= dens_pep_/I2
    # calculate density profile
    nF= np.shape(dens_pep)[0] # number of picked frames
    Lx,Ly,Lz= u.dimensions[:3]*CA
    iBeg_avg= round(nF*(1-ratio_avg))
    # 
    denM_pep= np.mean(dens_pep[iBeg_avg:,:],axis=0)

    dens= dens_pep
    zHf_pep, denHf_pep, denFit_pep, popt_pep= tanh_fit(zc,denM_pep)
    denCod_pep= popt_pep[0]
    denDil_pep= popt_pep[1]
    
    denDils_pep[i]= denDil_pep
    denCods_pep[i]= denCod_pep
    denHfs_pep[i,:]= denHf_pep
    denFits_pep[i,:]= denFit_pep
    popts_pep[i,:]= popt_pep
    
    print(f'{i} finished')

In [ ]:
# plot 
plt.rcParams.update({'font.size': 18})
plt.figure()
plt.plot(zHf_pep, denHf_pep*C,'.', color='#D00000', label=f'{seq}') #  {denCod_pep*C:.3g}, {denDil_pep*C:.3g}'
plt.plot(zHf_pep, denFit_pep*C, '--', linewidth=3, color='#D09000')
    
plt.xlim([0, 1.01*Lz/2])
plt.xlabel(r'$z$ [nm]')
plt.ylabel(r'$\rho_i(z)$ 10$^{-3}$ [nm$^{-3}$]')
plt.legend(frameon=False, fontsize=14)
plt.tight_layout()
# plt.savefig(folder+'/densZ_fit.png')

![](densZ_fit.png)

In [ ]:
# save variables
folderSave= folder0+'/results'
if not os.path.exists(folderSave):
    os.mkdir(folderSave) 
print(folderSave)  # build 'folderSave'

TempTag= f'{Temps[0]}-{Temps[-1]}'

with open(folderSave+'/dens'+TempTag+'.pkl', 'wb') as f: 
    pickle.dump([Temps, denDils_pep, denCods_pep], f) # 3

collect simulations under different temperatures

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

folder0C= ['WT',  'H12K'] # !
filenameC= ['dens260-290', 'dens200-260'] # ! 

In [ ]:
### load
NC= len(folder0C)
TempC= []
denDilC_pep= []
denCodC_pep= []
for i in range(NC):
    folderSave= folder0C[i]+'/results'
    with open(folderSave+'/'+filenameC[i]+'.pkl', 'rb') as f: 
        v1, v2, v3 = pickle.load(f)
    TempC.append(v1)
    denDilC_pep.append(v2)
    denCodC_pep.append(v3)

In [ ]:
### fitting
accu= 500
indC= [[0,0], [0,0]]

fitC_pep= []
TC_pep_cr= []
rhoC_pep_cr= []
for i in range(NC):
    denCod= denCodC_pep[i]
    denDil= denDilC_pep[i]
    Temp= TempC[i]
    i_beg= indC[i][0]
    i_end= indC[i][1]
    if i_end==0:
        i_end= len(Temp)
    fit, T_cr, rho_cr= find_Tc_from_diagram(denCod[i_beg:i_end], denDil[i_beg:i_end], Temp[i_beg:i_end], accu)
    fitC_pep.append(fit)
    TC_pep_cr.append(T_cr)
    rhoC_pep_cr.append(rho_cr)

The phase diagram of GY-23

![test image](phaseDiag_dens.png)